# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys_2 import g_key

#configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
#Reading from csv into dataframe using pandas
#####MAKE SURE TO GRAB THIS CSV FROM OTHER FOLDER USING RELATIVE PATHS
weatherdataframe = pd.read_csv("../WeatherPy/Weather CSV")
weatherdataframe.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mahébourg,-20.41,57.70,75.20,73,40,5.82,MU,1602862136
1,1,Tasiilaq,65.61,-37.64,42.80,93,75,3.36,GL,1602862403
2,2,Bluff,-46.60,168.33,41.16,69,20,1.28,NZ,1602862404
3,3,Geraldton,-28.77,114.60,64.40,72,15,17.22,AU,1602862405
4,4,Mataura,-46.19,168.86,33.39,94,33,1.95,NZ,1602862406


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#Store latitude and longitude in locations:
locations = weatherdataframe[["Lat", "Lng"]]

#Store humidity values. Fill NaN values and convert to float.

humidity = weatherdataframe["Humidity"].astype(float)



In [21]:
#Plot the heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = 10, point_radius = 1)

#Add the layer to the map
fig.add_layer(heat_layer)

#Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#Narrowed dataframe to find ideal weather condition: Temp
idealtemp_df = weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70)]

In [23]:
#Narrowed dataframe to find ideal weather condition: Wind Speed
idealwind_df = weatherdataframe.loc[weatherdataframe["Wind Speed"]<10]

In [24]:
#Narrowed dataframe to find ideal weather condition: Zero Cloudiness
idealwind_df = weatherdataframe.loc[weatherdataframe["Cloudiness"]==0]

In [25]:
#Combined all conditions into a single dataframe:

allconditions_df= weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70) & (weatherdataframe["Wind Speed"]<10) & (weatherdataframe["Cloudiness"]==0)]

allconditions_df







,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,Marzuq,14.40,46.47,74.75,20,0,4.81,YE,1602862446
50,50,Mercedes,-34.65,-59.43,78.01,63,0,8.01,AR,1602862424
55,55,Cape Town,-33.93,18.42,78.01,29,0,8.05,ZA,1602862461
85,85,Along,28.17,94.77,70.79,85,0,1.79,IN,1602862495
313,313,Abu Kamal,34.45,40.92,76.10,23,0,6.06,SY,1602862749
319,319,Saldanha,-33.01,17.94,77.00,33,0,9.17,ZA,1602862756
350,350,General Roca,-39.03,-67.58,78.80,39,0,8.05,AR,1602862791
441,441,Alice Springs,-23.70,133.88,75.20,31,0,4.70,AU,1602862825


In [26]:
allconditions_df.count()

Unnamed: 0    8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#New hotel dataframe
hotel_df = allconditions_df.reset_index()

In [28]:
#Creating new hotel name column for dataframe
hotel_df["Hotel Name"]= ""
hotel_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,41,41,Marzuq,14.40,46.47,74.75,20,0,4.81,YE,1602862446,
1,50,50,Mercedes,-34.65,-59.43,78.01,63,0,8.01,AR,1602862424,
2,55,55,Cape Town,-33.93,18.42,78.01,29,0,8.05,ZA,1602862461,
3,85,85,Along,28.17,94.77,70.79,85,0,1.79,IN,1602862495,
4,313,313,Abu Kamal,34.45,40.92,76.10,23,0,6.06,SY,1602862749,
5,319,319,Saldanha,-33.01,17.94,77.00,33,0,9.17,ZA,1602862756,
6,350,350,General Roca,-39.03,-67.58,78.80,39,0,8.05,AR,1602862791,
7,441,441,Alice Springs,-23.70,133.88,75.20,31,0,4.70,AU,1602862825,


In [32]:
#base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#creating parameters dictionary    
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Get coordinate data type from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    #Get name data from df
    city = row["City"]

    # Add keyword to params dict
    params["location"] = f"{latitude},{longitude}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Nearby hotel in {city}: {results[0]['name']}.")
        
        #Putting this iterated info into a dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    #Break
    time.sleep(1)


Retrieving Results for Index 0: Marzuq.
Missing field/result... skipping.
Retrieving Results for Index 1: Mercedes.
Missing field/result... skipping.
Retrieving Results for Index 2: Cape Town.
Missing field/result... skipping.
Retrieving Results for Index 3: Along.
Missing field/result... skipping.
Retrieving Results for Index 4: Abu Kamal.
Missing field/result... skipping.
Retrieving Results for Index 5: Saldanha.
Missing field/result... skipping.
Retrieving Results for Index 6: General Roca.
Missing field/result... skipping.
Retrieving Results for Index 7: Alice Springs.
Missing field/result... skipping.


In [31]:
#Checking if hotels printed
hotel_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,41,41,Marzuq,14.40,46.47,74.75,20,0,4.81,YE,1602862446,
1,50,50,Mercedes,-34.65,-59.43,78.01,63,0,8.01,AR,1602862424,
2,55,55,Cape Town,-33.93,18.42,78.01,29,0,8.05,ZA,1602862461,
3,85,85,Along,28.17,94.77,70.79,85,0,1.79,IN,1602862495,
4,313,313,Abu Kamal,34.45,40.92,76.10,23,0,6.06,SY,1602862749,
5,319,319,Saldanha,-33.01,17.94,77.00,33,0,9.17,ZA,1602862756,
6,350,350,General Roca,-39.03,-67.58,78.80,39,0,8.05,AR,1602862791,
7,441,441,Alice Springs,-23.70,133.88,75.20,31,0,4.70,AU,1602862825,


In [14]:
#Plotting hotels -- found on stack overflow, or refer to gmaps activity day 3
# data_layer = gmaps.symbol_layer(
# cgr[['Lat','Long']], fill_color='green', stroke_color='green', scale=cgr['5/1/20']
# )

# fig = gmaps.figure()

# fig.add_layer(data_layer)

# fig

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map


# Display figure
